In [ ]:
!pip install ipywidgets==7.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes accelerate xformers einops langchain transformers sentence-transformers langchain-community pypdf chromadb

# **Part 1 and 2**
# Ali KHadangi 10
# Pouria Nazemi 10
# Mona Saghafi 10

In [ ]:
import os
import chromadb

from langchain_community.document_loaders import HuggingFaceDatasetLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

if device == 'cuda':
    print(torch.cuda.get_device_name(0))

In [ ]:
orig_model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(orig_model_path)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
text = "What is Mistral? write your answer in one line"
mistral_llm.invoke(text)

'What is Mistral? write your answer in one line.\nA: Mistral is a French wind that blows from the Mediterranean Sea towards the Atlantic Ocean.'

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2", #choose embedding model
    model_kwargs={"device": "cuda"},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# **Part 3**
# Ali KHadangi 10
# Pouria Nazemi 10
# Mona Saghafi 10

In [ ]:
pdf_folder_path = "."
documents = []

for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path,file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

text_splitter_rc = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents_rc = text_splitter_rc.split_documents(documents)
chunked_documents_rc

[Document(page_content='Classification Based Outlier \nDetection Techniques\nAli Khadangi', metadata={'source': './Classification Based Outlier Detection Techniques.pdf', 'page': 0}),
 Document(page_content='Introduction\nOutlier detection istheprocess ofdetecting outliers, oradata point that isfaraway\nfrom theaverage, and depending onwhat you aretrying toaccomplish, potentially\nremoving orresolving them from theanalysis toprevent anypotential skewing .\nClassification isused tolearn amodel (classifier) from aset oflabeled data\ninstances (training) and then, classify atest instance into one oftheclasses using the\nlearnt model (testing) .', metadata={'source': './Classification Based Outlier Detection Techniques.pdf', 'page': 1}),
 Document(page_content='General Methodology of Operation\nClassification based outlier detection techniques operate intwo phases :\nTraining -The training phase learns aclassifier using theavailable labeled\ntraining data .1\nTesting –This phase classifies

In [ ]:
vectorstore = Chroma.from_documents(documents=chunked_documents_rc, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()

# **Part 4**
# Ali KHadangi 10
# Pouria Nazemi 10
# Mona Saghafi 10

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


template = """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, and provide concise and relevant responses.
Mention the number of the source document when relevant.
If the answer cannot be deduced from the context, state that you don't know.
Context: {context}
Question: {question}
your Answer: """


prompt = PromptTemplate(
        template=template, input_variables=["context", "question"]
    )


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | mistral_llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("what is RBF?")

"Using the information contained in the context,\ngive a comprehensive answer to the question.\nRespond only to the question asked, and provide concise and relevant responses.\nMention the number of the source document when relevant.\nIf the answer cannot be deduced from the context, state that you don't know.\nContext: SUPPORT VECTOR MACHINES BASED\nSupport Vector Machines (SVMs) have been applied to\noutlier detection inthe one-class setting .For learning\ncomplex regions Kernels, such asradial basis function (RBF)\nkernel, can beused .Ifatest instance falls within thelearnt\nregion, itisdeclared asnormal ,else itisdeclared asan\noutlier .\nAvariant ofthe basic technique finds the smallest hyper\nsphere inthe kernel space, which contains alltraining\ninstances ,and then determines which side ofthat hyper -\nsphere does atest instance lie.\n\nSUPPORT VECTOR MACHINES BASED\nSupport Vector Machines (SVMs) have been applied to\noutlier detection inthe one-class setting .For learning\ncom